In [1]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
URL = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"
response = requests.get(URL)
result= response.json()
result.keys() 


dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [38]:
len(result["Trip"])

5

In [39]:
result["Trip"][0].keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum', 'transferCount'])

In [40]:
result["Trip"][0]["LegList"]["Leg"][0]["Origin"]["name"]


'Stockholm Centralstation'

In [41]:
result["Trip"][0]["LegList"]["Leg"][0]["Destination"]["name"]   

'Linköping Centralstation'

In [42]:
def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [43]:
# checks first trip keys
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum', 'transferCount'])

In [44]:
example_trip['Origin']

{'name': 'Stockholm Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
 'extId': '740000001',
 'lon': 18.058151,
 'lat': 59.330136,
 'routeIdx': 0,
 'prognosisType': 'PROGNOSED',
 'time': '11:53:00',
 'date': '2025-01-10',
 'minimumChangeDuration': 'PT20M'}

In [45]:
example_trip['Destination']

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 11,
 'prognosisType': 'PROGNOSED',
 'time': '15:57:00',
 'date': '2025-01-10',
 'minimumChangeDuration': 'PT15M'}

In [46]:
example_trip["Destination"]

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 11,
 'prognosisType': 'PROGNOSED',
 'time': '15:57:00',
 'date': '2025-01-10',
 'minimumChangeDuration': 'PT15M'}

In [47]:
example_trip["TripStatus"]

{'hintCode': 0}

In [48]:
example_trip["ServiceDays"]

[{'planningPeriodBegin': '2025-01-02',
  'planningPeriodEnd': '2025-06-15',
  'sDaysR': 'inte varje dag',
  'sDaysI': '10. jan t o m 16. maj 2025 fre; utom 28. mar, 18. apr, 2., 9. maj',
  'sDaysB': '008102040810204081020008100040000200000000'}]

In [49]:
example_trip["LegList"].keys()

dict_keys(['Leg'])

In [50]:
len(example_trip["LegList"]["Leg"])

2

In [51]:
example_trip["LegList"]["Leg"][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration', 'minimumChangeDuration'])

In [52]:
example_stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
example_stops

[{'name': 'Stockholm Centralstation',
  'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
  'extId': '740000001',
  'routeIdx': 0,
  'lon': 18.058151,
  'lat': 59.330136,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '11:53:00',
  'depDate': '2025-01-10',
  'depDir': 'Karlstad Centralstation',
  'minimumChangeDuration': 'PT20M'},
 {'name': 'Katrineholm Centralstation',
  'id': 'A=1@O=Katrineholm Centralstation@X=16208324@Y=58996591@U=1@L=740000166@',
  'extId': '740000166',
  'routeIdx': 1,
  'lon': 16.208324,
  'lat': 58.996591,
  'depTime': '12:50:00',
  'depDate': '2025-01-10',
  'arrTime': '12:50:00',
  'arrDate': '2025-01-10',
  'minimumChangeDuration': 'PT10M'},
 {'name': 'Hallsberg station',
  'id': 'A=1@O=Hallsberg station@X=15110391@Y=59066698@U=1@L=740000077@',
  'extId': '740000077',
  'routeIdx': 2,
  'lon': 15.110391,
  'lat': 59.066698,
  'arrTime': '13:25:00',
  'arrDate': '2025-01-10',
  'minimumChangeDuration': 'PT10M'}]

In [53]:
for stop in example_stops:
    print(stop['name'])


Stockholm Centralstation
Katrineholm Centralstation
Hallsberg station


In [54]:
[{stop.get("name"): stop.get("arrTime")} for stop in example_stops]

[{'Stockholm Centralstation': None},
 {'Katrineholm Centralstation': '12:50:00'},
 {'Hallsberg station': '13:25:00'}]

# Stop lookup API to find id

In [55]:
location = "göteborg"
url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
response = requests.get(url)
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [56]:
stop_locations = result["stopLocationOrCoordLocation"]
len(stop_locations)

10

In [57]:
stop_locations[0]

{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
   {'icon': {'res': 'prod_gen'}, 'cls': '4'},
   {'icon': {'res': 'prod_gen'}, 'cls': '8'},
   {'icon': {'res': 'prod_gen'}, 'cls': '16'},
   {'icon': {'res': 'prod_gen'}, 'cls': '64'},
   {'icon': {'res': 'prod_gen'}, 'cls': '128'},
   {'icon': {'res': 'prod_gen'}, 'cls': '256'}],
  'timezoneOffset': 60,
  'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1736483800@',
  'extId': '740098001',
  'name': 'GÖTEBORG',
  'lon': 11.973479,
  'lat': 57.708895,
  'weight': 16027,
  'products': 222,
  'meta': True,
  'minimumChangeDuration': 'PT26M'}}

In [58]:
stop_locations[0]["StopLocation"].keys()

dict_keys(['productAtStop', 'timezoneOffset', 'id', 'extId', 'name', 'lon', 'lat', 'weight', 'products', 'meta', 'minimumChangeDuration'])

In [59]:
stop_locations[0]["StopLocation"]["name"], stop_locations[0]["StopLocation"]["extId"]

('GÖTEBORG', '740098001')

In [60]:
print("Stops in Göteborg")
for stop in stop_locations:
    print(stop["StopLocation"]["name"], stop["StopLocation"]["extId"])

Stops in Göteborg
GÖTEBORG 740098001
Göteborg Centralstation 740000002
Göteborg Sävenäs lokstation 740016365
GÖTEBORG GAMLESTADEN 740098526
Göteborg Korsvägen 740015578
Göteborg Kungsportsplatsen 740016358
Göteborg Stenpiren 740072430
Göteborg Eketrägatan 740025624
Göteborg Vårväderstorget 740025707
Göteborg Axel Dahlströms torg 740025608


In [61]:
# :<50  -> left align in a 50-character wide field
print(f"{'Stop':<50} {'extId'}")
for stop_location in stop_locations:
    stop = stop_location["StopLocation"]
    print(f"{stop.get('name'):<50} {stop.get('extId')}")

Stop                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608


In [62]:
def access_id_from_location(location):
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        result = response.json()

        print(f"{'Name':<50} extId")

        for stop in result.get("stopLocationOrCoordLocation"):
            stop_data = next(iter(stop.values()))

            # returns None if extId doesn't exist
            if stop_data.get("extId"):
                print(f"{stop_data['name']:<50} {stop_data['extId']}")

    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")

access_id_from_location("malm")

Name                                               extId
Malm (Gullspång kn)                                740062061
Malmö Centralstation                               740000003
Malmö Triangeln station                            740001587
Malmö Hyllie station                               740001586
Malmö Svågertorp station                           740001546
MALMÖ                                              740098548
Malmö Persborg station                             740001486
Malmö Rosengård station                            740001621
Malmö Östervärn station                            740001483
Malmö Fosieby station                              740001553


## Time tables
- [ResRobot Timetables](https://www.trafiklab.se/api/trafiklab-apis/resrobot-v21/timetables/)

Here I use pandas to read the json data, I'll leave it to you to make it into a function and find out relevant information

#url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

response = requests.get(url)
results = response.json()
results.keys()

#  0.a Make a function for doing get request for time tables API in resrobot as we didn't have a function in the lecture notes.


In [63]:
# korsvägen
stop_id=740015578
url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

response = requests.get(url)
results = response.json()
results.keys()

dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId'])

In [64]:
import pandas as pd 

df_timetable = pd.DataFrame(results["Departure"])
df_timetable.head()

,JourneyDetailRef,JourneyStatus,ProductAtStop,Product,Notes,name,type,stop,stopid,stopExtId,lon,lat,time,date,reachable,direction,directionFlag
0,{'ref': '1|369|0|1|10012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss X4,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,11:31:00,2025-01-10,True,Kungälv resecentrum,2
1,{'ref': '1|9947|1|1|10012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 61,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,11:31:00,2025-01-10,True,Göteborg Heden,2
2,{'ref': '1|124700|3|1|10012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 6,ST,Göteborg Scandinavium,A=1@O=Göteborg Scandinavium@X=11985875@Y=57700...,740025681,11.985875,57.700445,11:31:00,2025-01-10,True,Kortedala Aprilgatan (Göteborg kn),1
3,{'ref': '1|190|1|1|10012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss X4,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,11:32:00,2025-01-10,True,Höga hallar (Härryda kn),1
4,{'ref': '1|90380|2|1|10012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 2,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,11:32:00,2025-01-10,True,Mölndals Innerstad,1


In [65]:
df_timetable.columns

Index(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product',
       'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat',
       'time', 'date', 'reachable', 'direction', 'directionFlag'],
      dtype='object')

In [66]:
df_timetable[["name", "direction", "time"]]


,name,direction,time
0,Länstrafik - Buss X4,Kungälv resecentrum,11:31:00
1,Länstrafik - Buss 61,Göteborg Heden,11:31:00
2,Länstrafik - Spårväg 6,Kortedala Aprilgatan (Göteborg kn),11:31:00
3,Länstrafik - Buss X4,Höga hallar (Härryda kn),11:32:00
4,Länstrafik - Spårväg 2,Mölndals Innerstad,11:32:00
...,...,...,...
173,Länstrafik - Buss X4,Kungälv resecentrum,12:31:00
174,Länstrafik - Buss 61,Göteborg Heden,12:31:00
175,Länstrafik - Spårväg 2,Mölndals Innerstad,12:31:00
176,Länstrafik - Buss 18,Kallebäck (Göteborg kn),12:31:00


In [67]:
df_timetable_cleaned = df_timetable[["name", "stop", "lon", "lat", "direction", "date", "time"]]
df_timetable_cleaned


,name,stop,lon,lat,direction,date,time
0,Länstrafik - Buss X4,Göteborg Korsvägen,11.986918,57.696742,Kungälv resecentrum,2025-01-10,11:31:00
1,Länstrafik - Buss 61,Göteborg Korsvägen,11.986918,57.696742,Göteborg Heden,2025-01-10,11:31:00
2,Länstrafik - Spårväg 6,Göteborg Scandinavium,11.985875,57.700445,Kortedala Aprilgatan (Göteborg kn),2025-01-10,11:31:00
3,Länstrafik - Buss X4,Göteborg Korsvägen,11.986918,57.696742,Höga hallar (Härryda kn),2025-01-10,11:32:00
4,Länstrafik - Spårväg 2,Göteborg Korsvägen,11.986918,57.696742,Mölndals Innerstad,2025-01-10,11:32:00
...,...,...,...,...,...,...,...
173,Länstrafik - Buss X4,Göteborg Korsvägen,11.986918,57.696742,Kungälv resecentrum,2025-01-10,12:31:00
174,Länstrafik - Buss 61,Göteborg Korsvägen,11.986918,57.696742,Göteborg Heden,2025-01-10,12:31:00
175,Länstrafik - Spårväg 2,Göteborg Scandinavium,11.985875,57.700445,Mölndals Innerstad,2025-01-10,12:31:00
176,Länstrafik - Buss 18,Göteborg Korsvägen,11.986918,57.696742,Kallebäck (Göteborg kn),2025-01-10,12:31:00


In [68]:
df_timetable_cleaned["name"].value_counts()

name
Länstrafik - Spårväg 6    25
Länstrafik - Spårväg 2    24
Länstrafik - Spårväg 8    24
Länstrafik - Buss X4      17
Länstrafik - Buss 18      16
Länstrafik - Buss 61      13
Länstrafik - Spårväg 5    12
Länstrafik - Buss 63      12
Länstrafik - Spårväg 4    12
Länstrafik - Buss RÖD      8
Länstrafik - Buss 758      4
Länstrafik - Buss 100      3
Flygtransfer - Buss .      3
Länstrafik - Buss 101      2
Länstrafik - Buss 300      2
Expressbuss 601            1
Name: count, dtype: int64

In [69]:
#  Make a function for doing get request for time tables API in resrobot as we didn't have a function in the lecture notes.

def get_departures(stop_id):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        results = response.json()
        results.keys()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


In [72]:
import requests
import pandas as pd
import matplotlib.pyplot as plt



API_KEY = API_KEY = os.getenv("API_KEY")
BASE_URL = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"
STATION_ID = "740000001"  # ID for Göteborg Centralstationen

# Function to perform GET request for timetables
def get_departure_data(station_id, api_key):
    params = {
        'id': station_id,
        'format': 'json',
        'accessId': api_key
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

get_departure_data("740000001",API_KEY)

{'Departure': [{'JourneyDetailRef': {'ref': '1|9906|2|1|10012025'},
   'JourneyStatus': 'P',
   'ProductAtStop': {'icon': {'res': 'prod_gen'},
    'operatorInfo': {'name': 'Västtrafik',
     'nameS': '279',
     'nameN': '279',
     'nameL': 'Västtrafik',
     'id': '279'},
    'name': 'Länstrafik - Buss 61',
    'internalName': 'Länstrafik - Buss 61',
    'displayNumber': '61',
    'num': '61',
    'line': '61',
    'lineId': '1279506100001',
    'catOut': 'BLT',
    'catIn': 'BLT',
    'catCode': '7',
    'cls': '128',
    'catOutS': 'BLT',
    'catOutL': 'Länstrafik - Buss',
    'operatorCode': '279',
    'operator': 'Västtrafik',
    'admin': '279___',
    'matchId': '61;5061;83'},
   'Product': [{'icon': {'res': 'prod_gen'},
     'operatorInfo': {'name': 'Västtrafik',
      'nameS': '279',
      'nameN': '279',
      'nameL': 'Västtrafik',
      'id': '279'},
     'name': 'Länstrafik - Buss 61',
     'internalName': 'Länstrafik - Buss 61',
     'displayNumber': '61',
     'num': '